In [1]:
# load the packages
import json  
import re
import datetime as dt
from datetime import datetime
from collections import Counter
import time
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize  
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [2]:
df_2021 = pd.read_csv('/Users/asamitakeuchi/Bike/2021_cleaned_final.csv')

In [3]:
df_2021.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,week_day,month,time_start,trip_duration_mins
0,9,B9F73448DFBE0D45,classic_bike,2021-01-24 19:15:38,2021-01-24 19:22:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,member,0 days 00:07:13,Sunday,1,19:15:38,7.216667
1,10,457C7F4B5D3DA135,electric_bike,2021-01-23 12:57:38,2021-01-23 13:02:10,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,-87.696733,41.910435,-87.696890,member,0 days 00:04:32,Saturday,1,12:57:38,4.533333
2,11,57C750326F9FDABE,electric_bike,2021-01-09 15:28:04,2021-01-09 15:37:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,-87.696688,41.899180,-87.672178,casual,0 days 00:09:47,Saturday,1,15:28:04,9.783333
3,12,4D518C65E338D070,electric_bike,2021-01-09 15:28:57,2021-01-09 15:37:54,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,-87.696716,41.899149,-87.672177,casual,0 days 00:08:57,Saturday,1,15:28:57,8.950000
4,13,9D08A3AFF410474D,classic_bike,2021-01-24 15:56:59,2021-01-24 16:07:08,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,casual,0 days 00:10:09,Sunday,1,15:56:59,10.150000


In [4]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5076073 entries, 0 to 5076072
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   index               int64  
 1   ride_id             object 
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  trip_duration       object 
 15  week_day            object 
 16  month               int64  
 17  time_start          object 
 18  trip_duration_mins  float64
dtypes: float64(5), int64(2), object(12)
memory usage: 735.8+ MB


In [5]:
def check_if_null(df):
    null = df.isnull().sum()
    null_percent = (df.isnull().sum()/df.count()*100).round(2)
    null_values = pd.concat([null, null_percent], axis = 1)
    null_values.columns = ['Null Values', '%Null']
    return null_values

check_if_null(df_2021)

,Null Values,%Null
index,0,0.0
ride_id,0,0.0
rideable_type,0,0.0
started_at,0,0.0
ended_at,0,0.0
start_station_name,0,0.0
start_station_id,0,0.0
end_station_name,0,0.0
end_station_id,0,0.0
start_lat,0,0.0


## Merge Chicago wheathers dataset

In [6]:
chicago_weather = pd.read_csv('/Users/asamitakeuchi/Bike/Chicago_weather_2021.csv')
chicago_weather.head()

,Date,Average,Precipitation,New Snow
0,2021-01-01,27.0,1,1
1,2021-01-02,31.5,0,0
2,2021-01-03,29.5,1,1
3,2021-01-04,27.0,0,0
4,2021-01-05,33.0,0,0


In [7]:
chicago_weather.dtypes

Date              object
Average          float64
Precipitation      int64
New Snow           int64
dtype: object

In [8]:
c_date = pd.to_datetime(chicago_weather['Date'])
chicago_weather['Date'] = c_date.dt.date

date = pd.to_datetime(df_2021['started_at'])
df_2021['Date']=date.dt.date
df_2021['Date'].head()

0    2021-01-24
1    2021-01-23
2    2021-01-09
3    2021-01-09
4    2021-01-24
Name: Date, dtype: object

In [9]:
df_2021_weather_added = pd.merge(df_2021, chicago_weather, how='left', left_on='Date', right_on='Date')
df_2021_weather_added[:20]

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,...,member_casual,trip_duration,week_day,month,time_start,trip_duration_mins,Date,Average,Precipitation,New Snow
0,9,B9F73448DFBE0D45,classic_bike,2021-01-24 19:15:38,2021-01-24 19:22:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,...,member,0 days 00:07:13,Sunday,1,19:15:38,7.216667,2021-01-24,30.5,1,0
1,10,457C7F4B5D3DA135,electric_bike,2021-01-23 12:57:38,2021-01-23 13:02:10,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,...,member,0 days 00:04:32,Saturday,1,12:57:38,4.533333,2021-01-23,20.0,0,0
2,11,57C750326F9FDABE,electric_bike,2021-01-09 15:28:04,2021-01-09 15:37:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,...,casual,0 days 00:09:47,Saturday,1,15:28:04,9.783333,2021-01-09,32.5,0,0
3,12,4D518C65E338D070,electric_bike,2021-01-09 15:28:57,2021-01-09 15:37:54,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,...,casual,0 days 00:08:57,Saturday,1,15:28:57,8.950000,2021-01-09,32.5,0,0
4,13,9D08A3AFF410474D,classic_bike,2021-01-24 15:56:59,2021-01-24 16:07:08,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,...,casual,0 days 00:10:09,Sunday,1,15:56:59,10.150000,2021-01-24,30.5,1,0
5,14,49FCE1F8598F12C6,electric_bike,2021-01-22 15:15:28,2021-01-22 15:36:01,California Ave & Cortez St,17660,Wells St & Elm St,KA1504000135,41.900373,...,member,0 days 00:20:33,Friday,1,15:15:28,20.550000,2021-01-22,21.0,0,0
6,15,0FEED5C2C8749A1C,classic_bike,2021-01-05 10:33:12,2021-01-05 10:39:12,California Ave & Cortez St,17660,Sacramento Blvd & Franklin Blvd,KA1504000113,41.900363,...,member,0 days 00:06:00,Tuesday,1,10:33:12,6.000000,2021-01-05,33.0,0,0
7,16,E276FD43BDED6420,classic_bike,2021-01-30 11:59:16,2021-01-30 12:03:44,California Ave & Cortez St,17660,Western Ave & Walton St,KA1504000103,41.900363,...,member,0 days 00:04:28,Saturday,1,11:59:16,4.466667,2021-01-30,31.5,1,1
8,17,88BFCF66C2D585EC,electric_bike,2021-01-27 07:27:09,2021-01-27 07:45:32,California Ave & Cortez St,17660,Damen Ave & Clybourn Ave,13271,41.900313,...,member,0 days 00:18:23,Wednesday,1,07:27:09,18.383333,2021-01-27,20.5,0,0
9,18,8BD6F6510F5C8BD2,electric_bike,2021-01-15 08:54:41,2021-01-15 09:11:46,California Ave & Cortez St,17660,Damen Ave & Clybourn Ave,13271,41.900356,...,member,0 days 00:17:05,Friday,1,08:54:41,17.083333,2021-01-15,34.5,1,0


In [10]:
#rename 'Average', 'Precipitation', and 'New Snow' columns
df_2021_weather_added = df_2021_weather_added.rename(columns={'Average':'avg_temp','Precipitation':'rain','New Snow':'new_snow'})
df_2021_weather_added.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5076073 entries, 0 to 5076072
Data columns (total 23 columns):
 #   Column              Dtype  
---  ------              -----  
 0   index               int64  
 1   ride_id             object 
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  trip_duration       object 
 15  week_day            object 
 16  month               int64  
 17  time_start          object 
 18  trip_duration_mins  float64
 19  Date                object 
 20  avg_temp            float64
 21  rain                int64  
 22  new_snow            int64  
dtypes: float64(6), int64(4), object(13)
memory usage: 929.5+ 

# Create New Features

In [11]:
#create datetime features
end_dt = pd.to_datetime(df_2021_weather_added['ended_at'])
df_2021_weather_added['hour_end'] = end_dt.dt.hour
start_dt = pd.to_datetime(df_2021_weather_added['started_at'])
#start hours
df_2021_weather_added['hour_start'] = start_dt.dt.hour
#start & end minutes
df_2021_weather_added['minute_start'] = start_dt.dt.minute
df_2021_weather_added['minute_end'] = end_dt.dt.minute
#start & end day
df_2021_weather_added['day_start'] = start_dt.dt.day
df_2021_weather_added['day_end'] = end_dt.dt.day
#start & end month
df_2021_weather_added['month_start'] = start_dt.dt.month
df_2021_weather_added['month_end'] = end_dt.dt.month


df_2021_weather_added[:20]

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,...,rain,new_snow,hour_end,hour_start,minute_start,minute_end,day_start,day_end,month_start,month_end
0,9,B9F73448DFBE0D45,classic_bike,2021-01-24 19:15:38,2021-01-24 19:22:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,...,1,0,19,19,15,22,24,24,1,1
1,10,457C7F4B5D3DA135,electric_bike,2021-01-23 12:57:38,2021-01-23 13:02:10,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,...,0,0,13,12,57,2,23,23,1,1
2,11,57C750326F9FDABE,electric_bike,2021-01-09 15:28:04,2021-01-09 15:37:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,...,0,0,15,15,28,37,9,9,1,1
3,12,4D518C65E338D070,electric_bike,2021-01-09 15:28:57,2021-01-09 15:37:54,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,...,0,0,15,15,28,37,9,9,1,1
4,13,9D08A3AFF410474D,classic_bike,2021-01-24 15:56:59,2021-01-24 16:07:08,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,...,1,0,16,15,56,7,24,24,1,1
5,14,49FCE1F8598F12C6,electric_bike,2021-01-22 15:15:28,2021-01-22 15:36:01,California Ave & Cortez St,17660,Wells St & Elm St,KA1504000135,41.900373,...,0,0,15,15,15,36,22,22,1,1
6,15,0FEED5C2C8749A1C,classic_bike,2021-01-05 10:33:12,2021-01-05 10:39:12,California Ave & Cortez St,17660,Sacramento Blvd & Franklin Blvd,KA1504000113,41.900363,...,0,0,10,10,33,39,5,5,1,1
7,16,E276FD43BDED6420,classic_bike,2021-01-30 11:59:16,2021-01-30 12:03:44,California Ave & Cortez St,17660,Western Ave & Walton St,KA1504000103,41.900363,...,1,1,12,11,59,3,30,30,1,1
8,17,88BFCF66C2D585EC,electric_bike,2021-01-27 07:27:09,2021-01-27 07:45:32,California Ave & Cortez St,17660,Damen Ave & Clybourn Ave,13271,41.900313,...,0,0,7,7,27,45,27,27,1,1
9,18,8BD6F6510F5C8BD2,electric_bike,2021-01-15 08:54:41,2021-01-15 09:11:46,California Ave & Cortez St,17660,Damen Ave & Clybourn Ave,13271,41.900356,...,1,0,9,8,54,11,15,15,1,1


In [12]:
#New Features
#Distance of each users - calculate by using long and lat
import math
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(row):
    row = row.fillna(0.0)
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(row['start_lat'])
    lon1 = radians(row['start_lng'])
    lat2 = radians(row['end_lat'])
    lon2 = radians(row['end_lng'])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
   
    # Earth's radius in kilometers
    radius = 6371

    # Calculate the distance
    distance = radius * c
    return distance


df_2021_weather_added['distance'] = df_2021_weather_added.apply(haversine_distance, axis=1)

In [13]:
#Visualizaing the distance
df_2021_weather_added['distance']

0          2.032303
1          1.115287
2          2.032911
3          2.035544
4          2.032303
             ...   
5076068    0.864210
5076069    0.828035
5076070    0.782727
5076071    0.877700
5076072    0.593103
Name: distance, Length: 5076073, dtype: float64

In [14]:
df_2021_weather_added.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,...,new_snow,hour_end,hour_start,minute_start,minute_end,day_start,day_end,month_start,month_end,distance
0,9,B9F73448DFBE0D45,classic_bike,2021-01-24 19:15:38,2021-01-24 19:22:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,...,0,19,19,15,22,24,24,1,1,2.032303
1,10,457C7F4B5D3DA135,electric_bike,2021-01-23 12:57:38,2021-01-23 13:02:10,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,...,0,13,12,57,2,23,23,1,1,1.115287
2,11,57C750326F9FDABE,electric_bike,2021-01-09 15:28:04,2021-01-09 15:37:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,...,0,15,15,28,37,9,9,1,1,2.032911
3,12,4D518C65E338D070,electric_bike,2021-01-09 15:28:57,2021-01-09 15:37:54,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,...,0,15,15,28,37,9,9,1,1,2.035544
4,13,9D08A3AFF410474D,classic_bike,2021-01-24 15:56:59,2021-01-24 16:07:08,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,...,0,16,15,56,7,24,24,1,1,2.032303


## Feature encoding

In [15]:
#One-hot encoding for 'rideable_type','week_day' columns
one_hot_encoding_columns = ['rideable_type','week_day']
df_2021_encoded = pd.get_dummies(df_2021_weather_added, columns=one_hot_encoding_columns)

In [16]:
df_2021_encoded.reset_index()
df_2021_encoded.head()

,index,ride_id,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,rideable_type_classic_bike,rideable_type_docked_bike,rideable_type_electric_bike,week_day_Friday,week_day_Monday,week_day_Saturday,week_day_Sunday,week_day_Thursday,week_day_Tuesday,week_day_Wednesday
0,9,B9F73448DFBE0D45,2021-01-24 19:15:38,2021-01-24 19:22:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,...,1,0,0,0,0,0,1,0,0,0
1,10,457C7F4B5D3DA135,2021-01-23 12:57:38,2021-01-23 13:02:10,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,-87.696733,...,0,0,1,0,0,1,0,0,0,0
2,11,57C750326F9FDABE,2021-01-09 15:28:04,2021-01-09 15:37:51,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,-87.696688,...,0,0,1,0,0,1,0,0,0,0
3,12,4D518C65E338D070,2021-01-09 15:28:57,2021-01-09 15:37:54,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,-87.696716,...,0,0,1,0,0,1,0,0,0,0
4,13,9D08A3AFF410474D,2021-01-24 15:56:59,2021-01-24 16:07:08,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,...,1,0,0,0,0,0,1,0,0,0


In [81]:
#drop columns that aren't be used
drop_cols = (['start_station_id','end_station_id','ride_id','Date','start_station_name','end_station_name','trip_duration','started_at','ended_at','month','time_start','start_station_name','end_station_name'])
df_2021_dropped = df_2021_encoded.drop(drop_cols, axis=1).reset_index()
df_2021_dropped.head()

,level_0,index,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration_mins,avg_temp,rain,...,rideable_type_classic_bike,rideable_type_docked_bike,rideable_type_electric_bike,week_day_Friday,week_day_Monday,week_day_Saturday,week_day_Sunday,week_day_Thursday,week_day_Tuesday,week_day_Wednesday
0,0,9,41.900363,-87.696704,41.899181,-87.672200,member,7.216667,30.5,1,...,1,0,0,0,0,0,1,0,0,0
1,1,10,41.900406,-87.696733,41.910435,-87.696890,member,4.533333,20.0,0,...,0,0,1,0,0,1,0,0,0,0
2,2,11,41.900374,-87.696688,41.899180,-87.672178,casual,9.783333,32.5,0,...,0,0,1,0,0,1,0,0,0,0
3,3,12,41.900379,-87.696716,41.899149,-87.672177,casual,8.950000,32.5,0,...,0,0,1,0,0,1,0,0,0,0
4,4,13,41.900363,-87.696704,41.899181,-87.672200,casual,10.150000,30.5,1,...,1,0,0,0,0,0,1,0,0,0


In [82]:
df_2021_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5076073 entries, 0 to 5076072
Data columns (total 30 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   level_0                      int64  
 1   index                        int64  
 2   start_lat                    float64
 3   start_lng                    float64
 4   end_lat                      float64
 5   end_lng                      float64
 6   member_casual                object 
 7   trip_duration_mins           float64
 8   avg_temp                     float64
 9   rain                         int64  
 10  new_snow                     int64  
 11  hour_end                     int64  
 12  hour_start                   int64  
 13  minute_start                 int64  
 14  minute_end                   int64  
 15  day_start                    int64  
 16  day_end                      int64  
 17  month_start                  int64  
 18  month_end                    int64  
 19  

In [83]:
#Change the member_casual values to 0 and 1
df_2021_dropped['member_casual']= df_2021_dropped['member_casual'].apply(lambda x: 0 if x == 'casual' else 1)

In [84]:
df_2021_dropped.head()

,level_0,index,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration_mins,avg_temp,rain,...,rideable_type_classic_bike,rideable_type_docked_bike,rideable_type_electric_bike,week_day_Friday,week_day_Monday,week_day_Saturday,week_day_Sunday,week_day_Thursday,week_day_Tuesday,week_day_Wednesday
0,0,9,41.900363,-87.696704,41.899181,-87.672200,1,7.216667,30.5,1,...,1,0,0,0,0,0,1,0,0,0
1,1,10,41.900406,-87.696733,41.910435,-87.696890,1,4.533333,20.0,0,...,0,0,1,0,0,1,0,0,0,0
2,2,11,41.900374,-87.696688,41.899180,-87.672178,0,9.783333,32.5,0,...,0,0,1,0,0,1,0,0,0,0
3,3,12,41.900379,-87.696716,41.899149,-87.672177,0,8.950000,32.5,0,...,0,0,1,0,0,1,0,0,0,0
4,4,13,41.900363,-87.696704,41.899181,-87.672200,0,10.150000,30.5,1,...,1,0,0,0,0,0,1,0,0,0


In [86]:
df_2021_dropped.size

152282190

In [87]:
df_2021_dropped2 = df_2021_dropped.drop(['index','level_0'], axis=1).reset_index()
df_2021_dropped2[:5]

,index,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration_mins,avg_temp,rain,new_snow,...,rideable_type_classic_bike,rideable_type_docked_bike,rideable_type_electric_bike,week_day_Friday,week_day_Monday,week_day_Saturday,week_day_Sunday,week_day_Thursday,week_day_Tuesday,week_day_Wednesday
0,0,41.900363,-87.696704,41.899181,-87.672200,1,7.216667,30.5,1,0,...,1,0,0,0,0,0,1,0,0,0
1,1,41.900406,-87.696733,41.910435,-87.696890,1,4.533333,20.0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,2,41.900374,-87.696688,41.899180,-87.672178,0,9.783333,32.5,0,0,...,0,0,1,0,0,1,0,0,0,0
3,3,41.900379,-87.696716,41.899149,-87.672177,0,8.950000,32.5,0,0,...,0,0,1,0,0,1,0,0,0,0
4,4,41.900363,-87.696704,41.899181,-87.672200,0,10.150000,30.5,1,0,...,1,0,0,0,0,0,1,0,0,0


In [92]:
df_2021_dup_dropped = df_2021_dropped2.drop_duplicates(subset=subset_columns, keep='first', inplace=False,ignore_index=True)
print(df_2021_dup_dropped.size)

147206117


In [93]:
df_2021_dup_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5076073 entries, 0 to 5076072
Data columns (total 29 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   index                        int64  
 1   start_lat                    float64
 2   start_lng                    float64
 3   end_lat                      float64
 4   end_lng                      float64
 5   member_casual                int64  
 6   trip_duration_mins           float64
 7   avg_temp                     float64
 8   rain                         int64  
 9   new_snow                     int64  
 10  hour_end                     int64  
 11  hour_start                   int64  
 12  minute_start                 int64  
 13  minute_end                   int64  
 14  day_start                    int64  
 15  day_end                      int64  
 16  month_start                  int64  
 17  month_end                    int64  
 18  distance                     float64
 19  

In [95]:
df_2021_dup_dropped.to_csv('2021_prepped_new.csv'
             , header=True
             , index=False
             , chunksize=100000
             , encoding='utf-8')